In [1]:
import pandas as pd
import numpy as np
from io import StringIO
from sqlalchemy import create_engine
import sqlite3

In [2]:
basedata_df = pd.read_csv("shark_attacks.csv")

In [3]:
basedata_df = basedata_df.drop(columns = ["Unnamed: 17", "Unnamed: 18"])
basedata_df.head()

Case Number       Date    Year        Type    Country             Area  \
0  2018.06.25  25-Jun-18  2018.0     Boating        USA       California   
1  2018.06.18  18-Jun-18  2018.0  Unprovoked        USA          Georgia   
2  2018.06.09   9-Jun-18  2018.0     Invalid        USA           Hawaii   
3  2018.06.08   8-Jun-18  2018.0  Unprovoked  AUSTRALIA  New South Wales   
4  2018.06.04   4-Jun-18  2018.0    Provoked     MEXICO           Colima   

                         Location     Activity             Name Sex  Age  \
0     Oceanside, San Diego County     Paddling      Julie Wolfe   F   57   
1  St. Simon Island, Glynn County     Standing  Adyson McNeely    F   11   
2                    Habush, Oahu      Surfing      John Denges   M   48   
3              Arrawarra Headland      Surfing             male   M  NaN   
4                        La Ticla  Free diving   Gustavo Ramos    M  NaN   

                                              Injury Fatal (Y/N)  \
0  No injury to occupant, outrigger canoe and pad...           N   
1                         Minor injury to left thigh           N   
2       Injury to left lower leg from surfboard skeg           N   
3                          Minor injury to lower leg           N   
4  Lacerations to leg & hand shark PROVOKED INCIDENT           N   

            Time          Species          Investigator or Source  \
0          18h00      White shark                R. Collier, GSAF   
1  14h00  -15h00              NaN  K.McMurray, TrackingSharks.com   
2          07h45              NaN  K.McMurray, TrackingSharks.com   
3            NaN        2 m shark                  B. Myatt, GSAF   
4            NaN  Tiger shark, 3m                       A .Kipper   

                                        href formula  
0  http://sharkattackfile.net/spreadsheets/pdf_di...  
1  http://sharkattackfile.net/spreadsheets/pdf_di...  
2  http://sharkattackfile.net/spreadsheets/pdf_di...  
3  http://sharkattackfile.net/spreadsheets/pdf_di...  
4  http://sharkattackfile.net/spreadsheets/pdf_di...

In [4]:
df_cleaning = basedata_df.dropna()

In [5]:
basedata_df.describe()

Year
count  6300.000000
mean   1927.272381
std     281.116308
min       0.000000
25%    1942.000000
50%    1977.000000
75%    2005.000000
max    2018.000000

In [6]:
df_cleaning.describe()

Year
count  1423.000000
mean   1987.446943
std      94.361381
min       0.000000
25%    1978.000000
50%    2000.000000
75%    2009.000000
max    2018.000000

In [7]:
df_cleaning["Year"].min()

0.0

In [8]:
df_cleaning = df_cleaning[df_cleaning["Year"] != 0]

In [9]:
df_cleaning["Year"].min()

1868.0

In [10]:
df_cleaning["Year"].max()

2018.0

In [11]:
list(df_cleaning)

['Case Number',
 'Date',
 'Year',
 'Type',
 'Country',
 'Area',
 'Location',
 'Activity',
 'Name',
 'Sex',
 'Age',
 'Injury',
 'Fatal (Y/N)',
 'Time',
 'Species',
 'Investigator or Source',
 'href formula']

In [12]:
df_cleaning.Sex.unique()

array(['F', 'M', 'lli'], dtype=object)

In [13]:
df_cleaning = df_cleaning[df_cleaning["Sex"] != 'lli']
df_cleaning.Sex.unique()

array(['F', 'M'], dtype=object)

In [14]:
df_cleaning.Type.unique()

array(['Boating', 'Unprovoked', 'Provoked', 'Invalid', 'Sea Disaster'],
      dtype=object)

In [15]:
# strip all leading and trailing spaces of Country column
df_cleaning.Country = df_cleaning.Country.str.strip()
df_cleaning.Country.unique()

array(['USA', 'BRAZIL', 'AUSTRALIA', 'ENGLAND', 'COSTA RICA', 'MALDIVES',
       'SOUTH AFRICA', 'CUBA', 'BAHAMAS', 'INDONESIA', 'SPAIN', 'EGYPT',
       'NEW ZEALAND', 'FRENCH POLYNESIA', 'NEW CALEDONIA', 'ECUADOR',
       'THAILAND', 'REUNION', 'MEXICO', 'JAMAICA', 'ITALY',
       'PAPUA NEW GUINEA', 'RUSSIA', 'SEYCHELLES', 'VIETNAM',
       'MOZAMBIQUE', 'SCOTLAND', 'UNITED KINGDOM', 'TAIWAN', 'GUAM',
       'CROATIA', 'FIJI', 'TONGA', 'ST. MAARTIN', 'GRAND CAYMAN',
       'VANUATU', 'VENEZUELA', 'TURKS & CAICOS', 'OKINAWA',
       'MARSHALL ISLANDS', 'HONG KONG', 'JAPAN', 'SOUTH KOREA', 'ISRAEL',
       'MICRONESIA', 'COLUMBIA', 'NEW BRITAIN', 'PALAU',
       'SOLOMON ISLANDS', 'FEDERATED STATES OF MICRONESIA', 'GREECE',
       'BRITISH WEST INDIES', 'NORTH PACIFIC OCEAN', 'BERMUDA',
       'AMERICAN SAMOA', 'PACIFIC OCEAN', 'SENEGAL', 'ARGENTINA',
       'EL SALVADOR', 'IRAQ', 'PANAMA', 'INDIA'], dtype=object)

In [16]:
# strip all leading and trailing spaces of Area column
df_cleaning.Area = df_cleaning.Area.str.strip()
df_cleaning.Area.unique()

array(['California', 'Pernambuco', 'Florida', 'Queensland', 'Cornwall',
       'Cocos Island', 'Western Australia', 'Alifu Alifu Atoll',
       'Western Cape Province', 'Eastern Cape Province', 'Hawaii',
       'Victoria', 'New South Wales', 'Fernando de Noronha',
       'Holquin Province', 'South Australia', 'Massachusetts',
       'South Carolina', 'New Providence', 'Bali', 'Alicante Province',
       'Texas', 'North Carolina', 'Suez', 'North Island', 'Tuamotos',
       'Grand Terre', 'Galapagos Islands', 'Phuket', 'Tasmania',
       'Le Port', 'Rangiroa', 'Saint-Gilles-les-Bains', 'Bora Bora',
       'd’Étang-Salé', 'Abaco Islands', 'Alabama', 'Louisiana',
       'Delaware', 'Society Islands', 'South Island', 'North Province',
       'Oregon', 'Exuma Islands', 'Santa Cruz Island', 'Saint-Gilles',
       'Quintana Roo', 'Tuamotus', 'St. Catherine', 'Sinaloa', 'Sardinia',
       'Guerrero', 'Central Province',
       'Telyakovsky Bay, Khasan,  Primorsky Krai (Far East)', 'Praslin',
  

In [17]:
df_cleaning = df_cleaning[df_cleaning["Area"] != '0ºS, 142ºE']
df_cleaning = df_cleaning[df_cleaning["Area"] != 'Vava\x92u']
df_cleaning.Area.unique()

array(['California', 'Pernambuco', 'Florida', 'Queensland', 'Cornwall',
       'Cocos Island', 'Western Australia', 'Alifu Alifu Atoll',
       'Western Cape Province', 'Eastern Cape Province', 'Hawaii',
       'Victoria', 'New South Wales', 'Fernando de Noronha',
       'Holquin Province', 'South Australia', 'Massachusetts',
       'South Carolina', 'New Providence', 'Bali', 'Alicante Province',
       'Texas', 'North Carolina', 'Suez', 'North Island', 'Tuamotos',
       'Grand Terre', 'Galapagos Islands', 'Phuket', 'Tasmania',
       'Le Port', 'Rangiroa', 'Saint-Gilles-les-Bains', 'Bora Bora',
       'd’Étang-Salé', 'Abaco Islands', 'Alabama', 'Louisiana',
       'Delaware', 'Society Islands', 'South Island', 'North Province',
       'Oregon', 'Exuma Islands', 'Santa Cruz Island', 'Saint-Gilles',
       'Quintana Roo', 'Tuamotus', 'St. Catherine', 'Sinaloa', 'Sardinia',
       'Guerrero', 'Central Province',
       'Telyakovsky Bay, Khasan,  Primorsky Krai (Far East)', 'Praslin',
  

In [18]:
# strip all leading and trailing spaces of Location column
df_cleaning.Location = df_cleaning.Location.str.strip()
df_cleaning.Location.unique()

array(['Oceanside, San Diego County', 'Piedade Beach, Recife',
       'Cocoa Beach, Brevard  County', ..., 'Lake Macquarie',
       'Petrie Bight, Brisbane River', 'Ghat'], dtype=object)

In [19]:
# strip all leading and trailing spaces of Fatal column
df_cleaning["Fatal (Y/N)"] = df_cleaning["Fatal (Y/N)"].str.strip()
df_cleaning["Fatal (Y/N)"].unique()

array(['N', 'Y', 'M', '2017'], dtype=object)

In [20]:
# Drop rows with Fatal column values of 'M' and '2017'
df_cleaning = df_cleaning[df_cleaning["Fatal (Y/N)"] != 'M']
df_cleaning = df_cleaning[df_cleaning["Fatal (Y/N)"] != '2017']
df_cleaning["Fatal (Y/N)"].unique()

array(['N', 'Y'], dtype=object)

In [21]:
df_cleaning.describe()

Year
count  1417.000000
mean   1991.605505
std      23.553558
min    1868.000000
25%    1979.000000
50%    2000.000000
75%    2009.000000
max    2018.000000

In [22]:
# Drop rows in Age column that are not numeric
df_cleaning = df_cleaning[df_cleaning.Age.apply(lambda x: x.isnumeric())]
df_cleaning.Age.str.isnumeric().unique()

array([ True])

In [23]:
df_cleaning.Age.unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '13', '34',
       '50', '46', '48', '17', '28', '65', '73', '58', '36', '51', '61',
       '59', '42', '6', '23', '29', '39', '24', '12', '26', '11', '71',
       '43', '44', '14', '27', '62', '52', '38', '68', '16', '47', '63',
       '70', '9', '41', '53', '7', '66', '45', '74', '64', '8', '56',
       '49', '77', '84', '6½', '5'], dtype=object)

In [24]:
df_cleaning = df_cleaning[df_cleaning["Age"] != '6½']
df_cleaning.Age.unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '13', '34',
       '50', '46', '48', '17', '28', '65', '73', '58', '36', '51', '61',
       '59', '42', '6', '23', '29', '39', '24', '12', '26', '11', '71',
       '43', '44', '14', '27', '62', '52', '38', '68', '16', '47', '63',
       '70', '9', '41', '53', '7', '66', '45', '74', '64', '8', '56',
       '49', '77', '84', '5'], dtype=object)

In [25]:
df_cleaning.describe()

Year
count  1399.000000
mean   1991.598999
std      23.548174
min    1868.000000
25%    1979.000000
50%    2000.000000
75%    2009.000000
max    2018.000000

In [26]:
# strip all leading and trailing spaces of Injury column
df_cleaning.Injury = df_cleaning.Injury.str.strip()
df_cleaning.Injury.unique()

array(['No injury to occupant, outrigger canoe and paddle damaged',
       'FATAL', 'Lower left leg bitten',
       'Minor bite to hand by captive shark. PROVOKED INCIDENT',
       'Injured by teeth of a dead porbeagle shark he was tossing overboard.',
       'No injury, shark bit scuba gear',
       'No injury, knocked off board by shark', '5-inch cut to hand',
       'No injury, shark bit hole in ski',
       'Lacerations to left knee & lower leg',
       'Injuries to right leg & hand', 'Minor injuries to legs',
       'No injury, said to have been charged by sharks several times',
       'Minor injury, ankle grazed', 'Laceratons to right leg & foot',
       'Puncture wounds to leff foot & lower leg',
       'Minor injury to left forearm',
       'Severe lacerations to left thigh & knee', 'Right leg bitten',
       'Puncture wounds to left shoulder', 'Minor injury to foot',
       'Minor injuries', 'Lacerations to left hand', 'Abrasions',
       'Lacerations to left foot', 'Minor lac

In [27]:
df_cleaning.count()

Case Number               1399
Date                      1399
Year                      1399
Type                      1399
Country                   1399
Area                      1399
Location                  1399
Activity                  1399
Name                      1399
Sex                       1399
Age                       1399
Injury                    1399
Fatal (Y/N)               1399
Time                      1399
Species                   1399
Investigator or Source    1399
href formula              1399
dtype: int64

In [28]:
# strip all leading and trailing spaces of Activity column
df_cleaning.Activity = df_cleaning.Activity.str.strip()
df_cleaning.Activity.unique()

array(['Paddling', 'Swimming', 'Walking', 'Feeding sharks', 'Fishing',
       'Scuba diving', 'Surfing', 'Paddle-skiing',
       'Stand-Up Paddleboarding', 'Wading', 'Spearfishing',
       'Night bathing', 'Kayaking / Fishing', 'Snorkeling', 'SUP',
       'Kayaking', 'Body boarding', 'Scuba Diving', 'Surf skiing',
       'Surf fishing', 'Floating', 'SUP Foil boarding', 'Kite surfing',
       'Floating in tube', 'Standing', 'Diving', 'Teasing a shark',
       'Paddle boarding', 'Kayak Fishing', 'Surf-skiing',
       'Scallop diving on hookah', 'Playing in the water',
       'Body Boarding', 'Boogie boarding', 'Playing',
       'Standing / Snorkeling', 'Hand feeding sharks', 'Rowing',
       'Shark fishing', 'Body surfing', 'Kitesurfing', 'Kiteboarding',
       'Swimming / snorkeling', 'Diving for Abalone', 'Casting a net',
       'Cleaning fish', 'Boogie Boarding', 'Playing in the surf',
       'Kite boarding', "Fishing - 'tag & release'", 'Bodyboarding',
       'Swimming or boogie boar

In [29]:
df_cleaning.Activity.nunique()

323

In [30]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('fishing', case=False), 'Activity'] = 'Fishing'
df_cleaning.Activity.nunique()

269

In [31]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('scuba', case=False), 'Activity'] = 'Scuba Diving'
df_cleaning.Activity.nunique()

255

In [32]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('swimming', case=False), 'Activity'] = 'Swimming'
df_cleaning.Activity.nunique()

211

In [33]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('Foil boarding', case=False), 'Activity'] = 'Foil-Boarding'
df_cleaning.Activity.nunique()

211

In [34]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('paddle boarding', case=False), 'Activity'] = 'Paddle Boarding'
df_cleaning.loc[df_cleaning['Activity'].str.contains('Stand-Up Paddleboarding', case=False), 'Activity'] = 'Paddle Boarding'
df_cleaning.loc[df_cleaning['Activity'].str.contains('SUP'), 'Activity'] = 'Paddle Boarding'
df_cleaning.Activity.nunique()

208

In [35]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('bathing', case=False), 'Activity'] = 'Bathing'
df_cleaning.Activity.nunique()

203

In [36]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('snorkeling', case=False), 'Activity'] = 'Snorkeling'
df_cleaning.Activity.nunique()

199

In [37]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('floating', case=False), 'Activity'] = 'Floating'
df_cleaning.loc[df_cleaning['Activity'].str.contains('floatation', case=False), 'Activity'] = 'Floating'
df_cleaning.Activity.nunique()

192

In [38]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('splashing', case=False), 'Activity'] = 'Splashing'
df_cleaning.Activity.nunique()

190

In [39]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('skindiving', case=False), 'Activity'] = 'Skindiving'
df_cleaning.loc[df_cleaning['Activity'].str.contains('Skin diving', case=False), 'Activity'] = 'Skindiving'
df_cleaning.Activity.nunique()

187

In [40]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('abalone', case=False), 'Activity'] = 'Abalone Diving'
df_cleaning.Activity.nunique()

177

In [41]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('Paddleskiing', case=False), 'Activity'] = 'Paddle-skiing'
df_cleaning.Activity.nunique()

176

In [42]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('Surf skiing', case=False), 'Activity'] = 'Surf-skiing'
df_cleaning.Activity.nunique()

175

In [43]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('Bodyboarding', case=False), 'Activity'] = 'Body-boarding'
df_cleaning.loc[df_cleaning['Activity'].str.contains('Body Boarding', case=False), 'Activity'] = 'Body-boarding'
df_cleaning.Activity.nunique()

171

In [44]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('boogie boarding', case=False), 'Activity'] = 'Boogie Boarding'
df_cleaning.Activity.nunique()

169

In [45]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('kite', case=False), 'Activity'] = 'Kite-Surfing'
df_cleaning.Activity.nunique()

164

In [46]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('body surfing', case=False), 'Activity'] = 'Body-Surfing'
df_cleaning.Activity.nunique()

160

In [47]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('windsurfing', case=False), 'Activity'] = 'Wind-Surfing'
df_cleaning.Activity.nunique()

159

In [48]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('wading', case=False), 'Activity'] = 'Wading'
df_cleaning.Activity.nunique()

152

In [49]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('playing', case=False), 'Activity'] = 'Playing'
df_cleaning.Activity.nunique()

148

In [50]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('standing', case=False), 'Activity'] = 'Standing'
df_cleaning.Activity.nunique()

139

In [51]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('hookah', case=False), 'Activity'] = 'Hookah Diving'
df_cleaning.Activity.nunique()

136

In [52]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('free', case=False), 'Activity'] = 'Free Diving'
df_cleaning.Activity.nunique()

130

In [53]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('surfing ', case=False), 'Activity'] = 'Surfing'
df_cleaning.loc[df_cleaning['Activity'].str.contains(' surfing', case=False), 'Activity'] = 'Surfing'
df_cleaning.loc[df_cleaning['Activity'].str.contains('surfing,', case=False), 'Activity'] = 'Surfing'
df_cleaning.loc[df_cleaning['Activity'].str.contains('surfboard', case=False), 'Activity'] = 'Surfing'
df_cleaning.Activity.nunique()

109

In [54]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('Diving, feeding sharks', case=False), 'Activity'] = 'Diving'
df_cleaning.loc[df_cleaning['Activity'].str.contains('Diving in aquarium display tank', case=False), 'Activity'] = 'Diving'
df_cleaning.loc[df_cleaning['Activity'].str.contains('Diving off pier & treading water', case=False), 'Activity'] = 'Diving'
df_cleaning.loc[df_cleaning['Activity'].str.contains('Diving from the lugger San', case=False), 'Activity'] = 'Diving'
df_cleaning.loc[df_cleaning['Activity'].str.contains('Diving into water', case=False), 'Activity'] = 'Diving'
df_cleaning.loc[df_cleaning['Activity'].str.contains('Diving, but on the surface when bitten by the shark', case=False), 'Activity'] = 'Diving'
df_cleaning.Activity.nunique()

103

In [55]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('Pearl diving', case=False), 'Activity'] = 'Pearl Diving'
df_cleaning.Activity.nunique()

103

In [56]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('feeding sharks', case=False), 'Activity'] = 'Feeding Sharks'
df_cleaning.loc[df_cleaning['Activity'].str.contains('Feeding mullet to sharks', case=False), 'Activity'] = 'Feeding Sharks'
df_cleaning.Activity.nunique()

101

In [57]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('sittting', case=False), 'Activity'] = 'Sitting'
df_cleaning.loc[df_cleaning['Activity'].str.contains('sitting', case=False), 'Activity'] = 'Sitting'
df_cleaning.Activity.nunique()

99

In [58]:
list(df_cleaning.loc[df_cleaning['Activity'].str.contains('sitting', case=False), 'Activity'].unique())

['Sitting']

In [59]:
df_cleaning.Activity.unique()

array(['Paddling', 'Swimming', 'Walking', 'Feeding Sharks', 'Fishing',
       'Scuba Diving', 'Surfing', 'Paddle-skiing', 'Paddle Boarding',
       'Wading', 'Bathing', 'Snorkeling', 'Kayaking', 'Body-boarding',
       'Surf-skiing', 'Floating', 'Foil-Boarding', 'Kite-Surfing',
       'Standing', 'Diving', 'Teasing a shark', 'Hookah Diving',
       'Playing', 'Boogie Boarding', 'Rowing', 'Body-Surfing',
       'Abalone Diving', 'Casting a net', 'Cleaning fish', 'Free Diving',
       'Sitting', 'Washing sand off a speared fish', 'Wakeboarding',
       'Rescuing', 'Measuring sharks', 'Treading water', 'Jet skiing',
       "Accidentally stood on hooked shark's tail before attempting to gut it",
       'Jumping', 'Jumped into the water', 'Reviving a sedated shark',
       'Lifesaving drill', 'Holding onto an inflatable boat',
       'Chumming for white sharks',
       'Scallop diving (using surface-supplied air & a POD)',
       'Wreck / Technical diving', 'Wind-Surfing',
       "Lying pro

In [60]:
# df_shortened_years = df_cleaning.loc[df_cleaning['Year']>=2000]
# df_shortened_years.head()

In [61]:
# len(df_shortened_years.loc[df_shortened_years['Fatal (Y/N)'] == 'Y'])

In [62]:
min(df_cleaning.Year)

1868.0

In [63]:
max(df_cleaning.Year)

2018.0

In [64]:
df = df_cleaning
# df = df_shortened_years
df.head()

Case Number       Date    Year        Type    Country        Area  \
0     2018.06.25  25-Jun-18  2018.0     Boating        USA  California   
6   2018.06.03.a   3-Jun-18  2018.0  Unprovoked     BRAZIL  Pernambuco   
8   2018.05.26.b  26-May-18  2018.0  Unprovoked        USA     Florida   
10    2018.05.24  24-May-18  2018.0    Provoked  AUSTRALIA  Queensland   
13  2018.05.13.a  13-May-18  2018.0     Invalid    ENGLAND    Cornwall   

                        Location        Activity                    Name Sex  \
0    Oceanside, San Diego County        Paddling             Julie Wolfe   F   
6          Piedade Beach, Recife        Swimming  Jose Ernesto da Silva    M   
8   Cocoa Beach, Brevard  County         Walking               Cody High   M   
10               Cairns Aquarium  Feeding Sharks                    male   M   
13                Off Land's End         Fishing            Max Berryman   M   

   Age                                             Injury Fatal (Y/N)  \
0   57  No injury to occupant, outrigger canoe and pad...           N   
6   18                                              FATAL           Y   
8   15                              Lower left leg bitten           N   
10  32  Minor bite to hand by captive shark. PROVOKED ...           N   
13  21  Injured by teeth of a dead porbeagle shark he ...           N   

              Time           Species           Investigator or Source  \
0            18h00       White shark                 R. Collier, GSAF   
6   Late afternoon       Tiger shark   Diario de Pernambuco, 6/4/2018   
8            17h00    Bull shark, 6'   K.McMurray, TrackingSharks.com   
10         Morning   Grey reef shark           ABC.net.au ,05/24/2018   
13           08h15  Invalid incident  K. McMurray, TrackingSharks.com   

                                         href formula  
0   http://sharkattackfile.net/spreadsheets/pdf_di...  
6   http://sharkattackfile.net/spreadsheets/pdf_di...  
8   http://sharkattackfile.net/spreadsheets/pdf_di...  
10  http://sharkattackfile.net/spreadsheets/pdf_di...  
13  http://sharkattackfile.net/spreadsheets/pdf_di...

In [65]:
columns = df.columns 
columns = [i.replace(' ', '_') for i in columns]
columns

['Case_Number',
 'Date',
 'Year',
 'Type',
 'Country',
 'Area',
 'Location',
 'Activity',
 'Name',
 'Sex',
 'Age',
 'Injury',
 'Fatal_(Y/N)',
 'Time',
 'Species',
 'Investigator_or_Source',
 'href_formula']

## Export Dataframe as CSV

In [66]:
# df.to_csv("shark_data_cleaned.csv", index=False)

## Export to Database

In [67]:
engine = create_engine('sqlite:///shark.sqlite', echo=False)

In [68]:
df.columns = columns
df.to_sql('shark_data_cleaned', con=engine, if_exists='replace', index_label=None)
# engine.execute("SELECT * FROM shark_data_cleaned").fetchall()

In [69]:
#connect to the database
conn = sqlite3.connect('shark.sqlite')
c = conn.cursor()

c.executescript('''

    BEGIN TRANSACTION;
    ALTER TABLE shark_data_cleaned RENAME TO shark_data_cleaned_OLD;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE shark_data_cleaned ("id" BIGINT PRIMARY KEY NOT NULL,
                            "Case_Number" TEXT,
                            "Date" TEXT,
                            "Year"	FLOAT,
                            "Type"	TEXT,
                            "Country"	TEXT,
                            "Area"	TEXT,
                            "Location"	TEXT,
                            "Activity"	TEXT,
                            "Name"	TEXT,
                            "Sex"	TEXT,
                            "Age"	TEXT,
                            "Injury"	TEXT,
                            "Fatality"	TEXT,
                            "Time"	TEXT,
                            "Species"	TEXT,
                            "Investigator_or_Source"	TEXT,
                            "href_formula"	TEXT);

    INSERT INTO shark_data_cleaned SELECT * FROM shark_data_cleaned_OLD;

    DROP TABLE shark_data_cleaned_OLD;
    COMMIT TRANSACTION;
    ''')

#close out the connection
c.close()
conn.close()